# Advanced state

Draive provides a dedicated base class for defining application state (`State`) and serialized models (`DataModel`). While basic usage is quite straight forward, it hides a lot of customization and features to explore when needed. Both `State` and `DataModel` share most of the functionalities except the elements related to serialization and schema generation. We will focus on the `DataModel` as it has more details to cover.

## Dict conversion

First interesting feature not visible at a glance is an ability to convert from and to a dictionary populated with basic values.

In [1]:
from typing import Any

from draive import State


class DictConverted(State):
    name: str
    value: int

# all of this applies to the `DataModel` as well
dict_converted: DictConverted = DictConverted(name="converted", value=42)
dict_converted_as_dict: dict[str, Any] = dict_converted.as_dict()
dict_converted_from_dict: DictConverted = DictConverted.from_dict(dict_converted_as_dict)

## Mutations

Both `State` and `DataModel` are immutable by default. Attempting to change any of its fields value will result in both linting and runtime errors. The only valid method to apply a mutation is through a copy. There is a dedicated method to perform a mutating copy operation:

In [2]:
class Mutable(State):
    identifier: str
    value: int


initial: Mutable = Mutable(
    identifier="pre",
    value=42,
)
updated: Mutable = initial.updated(identifier="post")
final: Mutable = initial.updated(value=21)

print("initial", initial)
print("updated", updated)
print("final", final)

initial {'identifier': 'pre', 'value': 42}
updated {'identifier': 'post', 'value': 42}
final {'identifier': 'pre', 'value': 21}


## JSON conversion

Each `DataModel` can be serialized using JSON format. Current implementation uses an intermediate step of conversion from/to a dict using the methods described above.

In [3]:
from draive import DataModel


class JSONConverted(DataModel):
    name: str
    value: int


json_converted: JSONConverted = JSONConverted(name="converted", value=42)
json_converted_as_json: str = json_converted.as_json()
json_converted_from_json: JSONConverted = JSONConverted.from_json(json_converted_as_json)

## Model schema

Each `DataModel` has associated schema which is generated using type annotations of the class fields. Models have an ability to generate a JSON-schema compatible description:

In [4]:
class BasicModel(DataModel):
    field: str

print(BasicModel.json_schema(indent=2))

{
  "type": "object",
  "properties": {
    "field": {
      "type": "string"
    }
  },
  "required": [
    "field"
  ]
}


Additionally each model can generate a simplified schema description. This can be useful when requesting LLM structured data generation in some cases. Let's have a look:

In [5]:
print(BasicModel.simplified_schema(indent=2))

{
  "field": "string"
}


Field schema can be altered by using per field customization if needed.

## Field customization

Each field defined within the class can be customized by using a dedicated default value called `Field`. This allows to change the field validation, serialization or other elements.

### Alias and description

Fields can have aliases which can be used when initializing or modifying an instance as alternative names. Aliases can be also used for serialization. When generating a specification, aliases are always used instead of regular field names. Besides the alias, each field can also have a description which is also included in the schema:

In [6]:
from draive import Field


class CustomizedSchemaModel(DataModel):
    described: int = Field(description="Field description")
    aliased: str = Field(alias="field_alias")


print(CustomizedSchemaModel.json_schema(indent=2))
print("---")
print(CustomizedSchemaModel.simplified_schema(indent=2))

{
  "type": "object",
  "properties": {
    "described": {
      "type": "integer",
      "description": "Field description"
    },
    "aliased": {
      "type": "string"
    }
  },
  "required": [
    "described",
    "aliased"
  ]
}
---
{
  "described": "integer(Field description)",
  "aliased": "string"
}


### Default values

Fields can have regular default values instead of a dedicated `Field` default. When using `Field` you can still provide a default value and you can also define default value factory instead. When default value is not defined through the `Field` the `Field` itself does not serve a role of a default value.

In [7]:
class CustomizedDefaultsModel(DataModel):
    default: int = 42
    field_default: int = Field(default=21)
    field_default_factory: str = Field(default_factory=str)

### Verification

Each field is automatically validated based on its type annotation. However if a field is required to have any additional validation you can provide an appropriate function to execute in addition to regular validation.

In [8]:
def verifier(value: int) -> None:
    if value < 0:
        raise ValueError("Value can't be less than zero!")


class VerifiedModel(DataModel):
    value: int = Field(verifier=verifier)

### Validation

When automatically generated validation is not suitable for your specific case you can override it to provide custom validation function for each field. When that happens you are taking the full responsibility of ensuring proper value is used for a given field.

In [9]:
from draive import ParameterValidationContext, ParameterValidationError


def validator(
    value: Any,
    context: ParameterValidationContext,
) -> int:
    if isinstance(value, int):
        return value

    else:
        raise ParameterValidationError.invalid_type(
            expected=int,
            received=type(value),
            context=context,
        )


class ValidatedModel(DataModel):
    value: int = Field(validator=validator)

### Schema specification

Similarly to validation type schema specification is also automatically generated and can also be replaced with any custom schema specification for any given field.

In [10]:
class CustomizedSpecificationModel(DataModel):
    value: int = Field(specification={"type": "integer", "description":"Fully custom"})

### Conversion

Final element which can be specified is a function converting given field value to for the dict conversion or serialization of data. You can fully customize what will be the representation of the field when converting it to basic types. Make sure that deserializing (validating) the result back will not cause any issues.

In [11]:
def converter(value: int) -> str:
    return str(value)


class CustomizedConversionModel(DataModel):
    value: int = Field(converter=converter)

## Property paths

Additional advanced feature available is PropertyPath. Property path is an object which points to a given element inside the state/model object and can be used to retrieve it when needed. In order to create a property path you can use a special type variable `_`. It produces a fully typed `ParameterPath` instance, however it has to be type converted using the `path` method to ensure proper typing.

In [12]:
from draive import ParameterPath


class NestedPathModel(DataModel):
    values: list[int]


class PathModel(DataModel):
    nested: NestedPathModel
    value: int


path: ParameterPath[PathModel, list[int]] = PathModel.path(PathModel._.nested.values)
path_model_instance: PathModel = PathModel(
    value=21,
    nested=NestedPathModel(
        values=[42],
    ),
)
print(path(path_model_instance))

[42]


Property paths can be used not only as the getters for an instance that would came later. It also preserves the path as a string to be accessed later if needed.

In [13]:
print(path)

nested.values


Besides that paths can be also used to prepare per field requirements. We can simplify usage of paths in that case by avoiding the type conversion optional step:

In [14]:
from draive import ParameterRequirement

requirement: ParameterRequirement[PathModel] = ParameterRequirement[PathModel].equal(
    42,
    path=PathModel._.nested.values[0],
)

requirement.check(path_model_instance)

Requirements can be combined and examined. This can be used to provide an expressive interface for defining various filters.

In [15]:
print("lhs:", requirement.lhs)
print("operator:", requirement.operator)
print("rhs:", requirement.rhs)

combined_requirement: ParameterRequirement[PathModel] = requirement & ParameterRequirement[
    PathModel
].contained_in(
    [12, 21],
    path=PathModel._.value,
)

combined_requirement.check(path_model_instance)

lhs: nested.values[0]
operator: equal
rhs: 42
